In [163]:
import pandas as pd
from sklearn.utils import shuffle
import numpy as np
import xgboost as xgb

xgb.set_config(verbosity=0)
import joblib


In [164]:
model_p = xgb.XGBRegressor(verbosity=0)
model_p.load_model("../../emotion_classifier/model_training/models/l2p_dance_model_Fs2_O.json")

model_a = xgb.XGBRegressor(verbosity=0)
model_a.load_model("../../emotion_classifier/model_training/models/l2a_dance_model_Fs2_O.json")

model_d = xgb.XGBRegressor(verbosity=0)
model_d.load_model("../../emotion_classifier/model_training/models/l2d_dance_model_Fs2_O.json")

scaler = joblib.load('../../emotion_classifier/model_training/datasets/scalers/standardizers/Fs2_B_O_S_DANCE_WALK_KIN_0.5sec.pkl') 

In [165]:
dataset = pd.read_csv('datasets/Fs3_B_O_DANCE_WALK_KIN_0.5sec.csv')
dataset.head()

,max_hand_distance,avg_l_hand_hip_distance,avg_r_hand_hip_distance,max_stride_length,avg_l_hand_chest_distance,avg_r_hand_chest_distance,avg_l_elbow_hip_distance,avg_r_elbow_hip_distance,avg_neck_chest_distance,avg_neck_rotation_w,...,r_foot_speed,neck_speed,l_hand_acceleration_magnitude,r_hand_acceleration_magnitude,l_foot_acceleration_magnitude,r_foot_acceleration_magnitude,neck_acceleration_magnitude,EMOTION_P,EMOTION_A,EMOTION_D
0,0.684605,0.268051,0.236534,0.478955,0.469381,0.455152,0.351238,0.336392,0.278741,0.011497,...,0.531111,0.677884,1.109843,0.766334,1.537713,1.062221,1.355767,-0.5,0.6,0.9
1,0.438911,0.265254,0.229588,0.283025,0.468163,0.452005,0.350459,0.333469,0.278750,0.011885,...,0.014436,0.021416,1.085836,0.779991,1.575538,1.049393,1.367237,-0.5,0.6,0.9
2,0.440199,0.266441,0.223630,0.309053,0.455287,0.439872,0.343657,0.325378,0.278739,0.017740,...,0.072819,0.191442,1.231576,1.117785,0.133194,0.126177,0.355767,-0.5,0.6,0.9
3,0.500120,0.386881,0.393727,0.452718,0.418828,0.440395,0.324313,0.340373,0.278729,0.033718,...,0.464108,0.605356,1.068333,1.038694,1.638293,1.021144,1.332992,-0.5,0.6,0.9
4,0.562528,0.447009,0.596923,0.488571,0.361697,0.522971,0.286162,0.430045,0.278747,0.041425,...,0.139125,0.207208,1.304347,1.077684,1.563165,0.917321,0.803440,-0.5,0.6,0.9


In [166]:
train_dataset = dataset.sample(frac=0.95, random_state=42)
test_dataset = dataset.drop(train_dataset.index)

print("No Training Samples:",train_dataset.shape[0])
print("No Test Samples:",test_dataset.shape[0])

train_dataset = shuffle(train_dataset)
test_dataset = shuffle(test_dataset)

No Training Samples: 38538
No Test Samples: 2028


In [167]:
train_emotions = pd.concat([train_dataset.pop(x) for x in ['EMOTION_P', 'EMOTION_A', 'EMOTION_D']], axis=1)
train_emotions_OG = train_emotions.copy()

test_emotions = pd.concat([test_dataset.pop(x) for x in ['EMOTION_P', 'EMOTION_A', 'EMOTION_D']], axis=1)
test_emotions_OG = test_emotions.copy()

In [168]:
train_emotions_p = model_p.predict(train_dataset)
train_emotions_a = model_a.predict(train_dataset)
train_emotions_d = model_d.predict(train_dataset)

rows = []
for i in range(len(train_dataset)):
    rows.append([train_emotions_p[i], train_emotions_a[i], train_emotions_d[i]])

train_emotions = pd.DataFrame(rows, columns=[
            "EMOTION_P", "EMOTION_A", "EMOTION_D"
         ])

train_emotions.head()

,EMOTION_P,EMOTION_A,EMOTION_D
0,0.170779,0.420426,0.074570
1,0.316093,0.395268,0.191429
2,0.290567,0.417218,0.108208
3,0.487052,0.309305,0.258323
4,0.279108,0.477474,0.098426


In [169]:
test_emotions_p = model_p.predict(test_dataset)
test_emotions_a = model_a.predict(test_dataset)
test_emotions_d = model_d.predict(test_dataset)

rows = []
for i in range(len(test_dataset)):
    rows.append([test_emotions_p[i], test_emotions_a[i], test_emotions_d[i]])

test_emotions = pd.DataFrame(rows, columns=[
            "EMOTION_P", "EMOTION_A", "EMOTION_D"
         ])

test_emotions.head()

,EMOTION_P,EMOTION_A,EMOTION_D
0,0.442142,0.469476,0.213168
1,-0.103089,0.413134,0.224867
2,0.276700,0.364156,0.130408
3,-0.013347,0.405878,0.231400
4,0.203966,0.427358,0.117642


In [170]:
def vae_loss(input_vols, output_vols):
    beta = 1e-7
    kl_loss = K.sum(-1 - K.log(K.exp(log_var)) + K.exp(log_var) + K.square(mu))/2
    return K.mean((input_vols-output_vols)**2) + beta*kl_loss

In [171]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import matplotlib.pyplot as plt

## Sampling Layer

In [172]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

## Encoder

In [173]:
latent_dim = 4

encoder_inputs = keras.Input(shape=(26, ))
x = layers.Dense(16, activation="relu")(encoder_inputs)
x = layers.Dense(8, activation="relu")(x)
x = layers.Dense(4, activation="relu")(x)

z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])

encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_37 (InputLayer)          [(None, 26)]         0           []                               
                                                                                                  
 dense_108 (Dense)              (None, 16)           432         ['input_37[0][0]']               
                                                                                                  
 dense_109 (Dense)              (None, 8)            136         ['dense_108[0][0]']              
                                                                                                  
 dense_110 (Dense)              (None, 4)            36          ['dense_109[0][0]']              
                                                                                            

## Decoder

In [174]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(4, activation="relu")(latent_inputs)
x = layers.Dense(8, activation="relu")(x)
x = layers.Dense(16, activation="relu")(x)

decoder_outputs = layers.Dense((26, )[0], activation="linear")(x)

decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_38 (InputLayer)       [(None, 4)]               0         
                                                                 
 dense_111 (Dense)           (None, 4)                 20        
                                                                 
 dense_112 (Dense)           (None, 8)                 40        
                                                                 
 dense_113 (Dense)           (None, 16)                144       
                                                                 
 dense_114 (Dense)           (None, 26)                442       
                                                                 
Total params: 646
Trainable params: 646
Non-trainable params: 0
_________________________________________________________________


## VAE

In [175]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            
            reconstruction = self.decoder(z)
            reconstruction_loss = keras.losses.mean_squared_error(data, reconstruction)
            
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            
            total_loss = (50 * reconstruction_loss) + (kl_loss)
            
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

## Train

In [ ]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(train_dataset, epochs=256, batch_size=5)

Epoch 1/256
7708/7708 [==============================] - 10s 1ms/step - loss: 8.7512 - reconstruction_loss: 0.1118 - kl_loss: 1.2754
Epoch 2/256
7708/7708 [==============================] - 9s 1ms/step - loss: 6.1713 - reconstruction_loss: 0.0983 - kl_loss: 1.2626
Epoch 3/256
7708/7708 [==============================] - 9s 1ms/step - loss: 6.1648 - reconstruction_loss: 0.0979 - kl_loss: 1.2628
Epoch 4/256
7708/7708 [==============================] - 9s 1ms/step - loss: 6.1931 - reconstruction_loss: 0.0976 - kl_loss: 1.2656
Epoch 5/256
7708/7708 [==============================] - 10s 1ms/step - loss: 6.1367 - reconstruction_loss: 0.0975 - kl_loss: 1.2649
Epoch 6/256
7708/7708 [==============================] - 9s 1ms/step - loss: 6.0765 - reconstruction_loss: 0.0975 - kl_loss: 1.2562
Epoch 7/256
7708/7708 [==============================] - 9s 1ms/step - loss: 6.1384 - reconstruction_loss: 0.0973 - kl_loss: 1.2534
Epoch 8/256
7708/7708 [==============================] - 9s 1ms/step - los

7708/7708 [==============================] - 10s 1ms/step - loss: 6.0608 - reconstruction_loss: 0.0920 - kl_loss: 1.4797
Epoch 63/256
7708/7708 [==============================] - 10s 1ms/step - loss: 6.0337 - reconstruction_loss: 0.0916 - kl_loss: 1.4589
Epoch 64/256
7708/7708 [==============================] - 10s 1ms/step - loss: 6.0437 - reconstruction_loss: 0.0912 - kl_loss: 1.4494
Epoch 65/256
7708/7708 [==============================] - 10s 1ms/step - loss: 6.0917 - reconstruction_loss: 0.0924 - kl_loss: 1.4674
Epoch 66/256
7708/7708 [==============================] - 9s 1ms/step - loss: 6.1503 - reconstruction_loss: 0.0944 - kl_loss: 1.3953
Epoch 67/256
7708/7708 [==============================] - 9s 1ms/step - loss: 6.1509 - reconstruction_loss: 0.0941 - kl_loss: 1.3778
Epoch 68/256
7708/7708 [==============================] - 9s 1ms/step - loss: 6.0701 - reconstruction_loss: 0.0937 - kl_loss: 1.4318
Epoch 69/256
7708/7708 [==============================] - 9s 1ms/step - loss: 

7708/7708 [==============================] - 9s 1ms/step - loss: 5.5501 - reconstruction_loss: 0.0753 - kl_loss: 1.7734
Epoch 124/256
7708/7708 [==============================] - 9s 1ms/step - loss: 5.5522 - reconstruction_loss: 0.0754 - kl_loss: 1.7616
Epoch 125/256
7708/7708 [==============================] - 10s 1ms/step - loss: 5.5246 - reconstruction_loss: 0.0754 - kl_loss: 1.7527
Epoch 126/256
7708/7708 [==============================] - 9s 1ms/step - loss: 5.5270 - reconstruction_loss: 0.0753 - kl_loss: 1.7634
Epoch 127/256
7708/7708 [==============================] - 9s 1ms/step - loss: 5.5199 - reconstruction_loss: 0.0753 - kl_loss: 1.7684
Epoch 128/256
7708/7708 [==============================] - 9s 1ms/step - loss: 5.5377 - reconstruction_loss: 0.0752 - kl_loss: 1.7701
Epoch 129/256
7708/7708 [==============================] - 9s 1ms/step - loss: 5.4825 - reconstruction_loss: 0.0752 - kl_loss: 1.7771
Epoch 130/256
7708/7708 [==============================] - 9s 1ms/step - lo

7708/7708 [==============================] - 9s 1ms/step - loss: 5.4788 - reconstruction_loss: 0.0735 - kl_loss: 1.8195
Epoch 185/256
7708/7708 [==============================] - 9s 1ms/step - loss: 5.4724 - reconstruction_loss: 0.0732 - kl_loss: 1.8248
Epoch 186/256
7708/7708 [==============================] - 9s 1ms/step - loss: 5.4713 - reconstruction_loss: 0.0733 - kl_loss: 1.8225
Epoch 187/256
7708/7708 [==============================] - 9s 1ms/step - loss: 5.4442 - reconstruction_loss: 0.0733 - kl_loss: 1.8250
Epoch 188/256
7708/7708 [==============================] - 9s 1ms/step - loss: 5.5055 - reconstruction_loss: 0.0733 - kl_loss: 1.8331
Epoch 189/256
7708/7708 [==============================] - 9s 1ms/step - loss: 5.4997 - reconstruction_loss: 0.0734 - kl_loss: 1.8254
Epoch 190/256
7708/7708 [==============================] - 9s 1ms/step - loss: 5.4867 - reconstruction_loss: 0.0735 - kl_loss: 1.8209
Epoch 191/256
7708/7708 [==============================] - 9s 1ms/step - los

## Test

In [ ]:
conv_dict = {
    (-0.5, 0.6, 0.9): "angry",
    (0.6, 0.5, 0.2): "happy",
    (-0.6, -0.3, -0.3): "sad",
    (-0.4, 0.25, -0.1): "disgusted",
    (-0.35, 0.7, -0.8): "afraid",
    (0.7, 0.2, 0.2): "pleased",
    (-0.5, -0.7, -0.25): "bored",
    (0.1, -0.7, -0.2): "tired",
    (0.6, -0.55, 0.1): "relaxed",
    (0.5, 0.7, 0.4): "excited",
    (-0.85, -0.1, -0.8): "miserable",
    (-0.3, -0.66, -0.7): "nervous",
    (0.9, -0.25, 0.65): "satisfied",   
}

colour_dict = {
    "angry": "crimson",
    "happy": "springgreen",
    "sad": "cornflowerblue",
    "disgusted": "darkorange"  ,
    "afraid": "gold",
    "pleased": "olive",
    "bored": "lightseagreen",
    "tired": "plum",
    "relaxed": "chocolate",
    "excited": "olivedrab",
    "miserable": "purple",
    "nervous": "lightslategray",
    "satisfied": "lightpink",   
}

def plot_label_clusters(vae, data, labels):
    # display a 2D plot of the digit classes in the latent space
    z_mean, _, _ = vae.encoder.predict(data)
    plt.figure(figsize=(12, 10))
    
    actual_labels = []
    for i in range(len(labels)):
        point_coords = (labels.iloc[i][0], labels.iloc[i][1], labels.iloc[i][2])

        actual_labels.append(colour_dict[conv_dict[point_coords]])

    
    plt.scatter(z_mean[:, 0], z_mean[:, 1], c=actual_labels)
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.show()

plot_label_clusters(vae, test_dataset, test_emotions_OG)
